In [1]:
import django 
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from sch.models import *
from sch.actions import *

In [2]:
slot = Slot.objects.get(slug__contains='2022-11-05-RS')
def resolveTurnaroundWithinDay (turnaroundSlot):
        emp = turnaroundSlot.employee
        if emp.evening_pref:
            wd = turnaroundSlot.workday
            sfta = turnaroundSlot.shift
        elif emp.evening_pref == False:
            wd = turnaroundSlot.workday.prevWD()
            sfta = Slot.objects.get(workday=wd,employee=emp)
        
        sfts = emp.shifts_available.all()
        print('shifts_available',sfts)
        noIncompatibles = []
        for sft in sfts:
            if not Slot.objects.incompatible_slots(workday=wd,shift=sft).filter(employee=emp).exists():
                noIncompatibles += [sft]
                print(sft, noIncompatibles)
        List = []
        for sft in noIncompatibles:
            checking = Slot.objects.filter(workday=wd,shift=sft)
            print(checking)
            if checking.exists():
                if sfta in checking.first().employee.shifts_available.all():
                    if not Slot.objects.incompatible_slots(workday=wd,shift=sfta).filter(employee=checking.first().employee).exists():
                        List += [sfta]  
        print(List)  
        if len(List) == 1:
            return List[0]  
        return List[random.randint(0,len(List)-1)]

In [3]:
from django.db.models import IntegerField 

In [4]:
emp = Employee.objects.get(name='Josh')
print("Avg Agg Shift Pref Score:",
      emp.agg_avg_shift_pref_score)
print("WeeklyHours, week 43:",
      emp.weekly_hours(2022,43), 'hrs')
print("WeeklyHours Percent:",
      emp.weekly_hours_perc(2022,43) * 100, "%")

Avg Agg Shift Pref Score: -0.1111111111111111
WeeklyHours, week 43: 40.0 hrs
WeeklyHours Percent: 160.0 %


In [5]:
emp.templated_days

<QuerySet [<ShiftTemplate: SatA:MI Template>, <ShiftTemplate: SunB:MI Template>, <ShiftTemplate: SatB:MI Template>, <ShiftTemplate: SunB:MI Template>]>

In [6]:
wds = Workday.objects.filter(date__year=2022,iweek=3).first()
wds.who_can_fill(Shift.objects.get(name='MI'))


<EmployeeManager [<Employee: Josh>, <Employee: Sabrina>, <Employee: Brianna>, <Employee: Esperanza>, <Employee: Elisia>, <Employee: Jaden>, <Employee: Trisha>, <Employee: Tiffany>, <Employee: David>, <Employee: Nicki>, <Employee: Leslie>]>

In [7]:
for sft in Shift.objects.filter(start__hour__gte=20):
    sft.group = 'XT'
    sft.save()

In [8]:
slts = Slot.objects.filter(shift__name='S',workday__date__year=2022,workday__iweek=43)

print(slts)

<SlotManager [<Slot: 221025-S Jaden>, <Slot: 221024-S Jaden>, <Slot: 221026-S Brianna>, <Slot: 221027-S Tiffany>, <Slot: 221028-S Esperanza>]>


In [38]:
slt = Slot.objects.filter(slug__icontains='2022-11-03').first()
incompat_slts = Slot.objects.filter(workday=slt.workday.prevWD(),shift__start__hour__lte=9)
slt

<Slot: 221103-7P Brianna>

In [39]:
def removePtoConflictSlots (year=None,week=None):
    if not year & week:
        allConflicts = Slot.objects.filter(is_turnaround=True) | Slot.objects.filter(is_preturnaround=True)
        for conflict in allConflicts:
            if conflict.employee.evening_pref:
                if conflict.shift.start.hour < 10:
                    print('Deleting %s' % conflict)
                    s = Slot.objects.filter(pk=conflict.pk).update(employee=None)
                    s.save()
            elif conflict.employee.evening_pref == False:
                if conflict.shift.start.hour > 10:
                    print('Deleting %s' % conflict)
                    s = Slot.objects.filter(pk=conflict.pk).update(employee=None)
                    s.save()
        

<ShiftPreferenceManager []>

In [52]:
allConflicts = Slot.objects.annotate(
                ptoConflict=Subquery(PtoRequest.objects.filter(
                    employee=OuterRef('employee'), 
                    workday=OuterRef('workday__date')).values('workday')[:1]))

ac = allConflicts.exclude(ptoConflict=None)
ac

<SlotManager [<Slot: 221127-RN Mary>, <Slot: 221128-RN Mary>, <Slot: 221129-RN Mary>, <Slot: 220916-7C Elisia>, <Slot: 220922-7C David>, <Slot: 220926-MI David>, <Slot: 221013-C Kristen>, <Slot: 221026-7C Tiffany>, <Slot: 221024-MI Brianna>, <Slot: 221018-7C Jaden>, <Slot: 221114-C Lynlee>, <Slot: 221115-PC Molly>]>

In [43]:
for emp in Employee.objects.all():
    if not emp.evening_pref:
        print(emp.name)

Josh
Molly
Doug
Brittanie
Esperanza
Carrie
Jaden
Lynlee
Carol
Trisha
Tiffany
Brianna S
Mary
Andrew
Sam
David
Nicki
Kristen
Shaun
Alex
Leslie
Kat
Bara
Erin
